係り元の文節と係り先の文節のテキストをタブ区切り形式ですべて抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [1]:
# https://qiita.com/segavvy/items/003e8b7e0f132f4fde1b

import re

class Morph:
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface # 表層形(surface)
        self.base    = base    # 基本形(base)
        self.pos     = pos     # 品詞(pos)
        self.pos1    = pos1    # 品詞細分類1(pos1)
        
    def __str__(self):
        '''オブジェクトの文字列表現'''
        return 'surface[{}]\tbase[{}]\tpos[{}]\tpos1[{}]'\
            .format(self.surface, self.base, self.pos, self.pos1)

In [15]:
class Chunk:
    def __init__(self):
        '''初期化'''
        self.morphs = []
        self.srcs   = []   # 係り元文節インデックス番号のリスト
        self.dst    = -1   # 係り先文節インデックス番号(初期値:-1, 係り先がない場合は-1のまま)
        
    def __str__(self):
        '''オブジェクトの文字列表現'''
        surface = ''
        for morph in self.morphs:
            surface += morph.surface
        return '{}\tsrcs{}\tdst[{}]'.format(surface, self.srcs, self.dst)
    
    def output_surface_wo_pct(self):
        surface = ''
        for morph in self.morphs:
            if morph.pos != '記号':
                surface += morph.surface
        return surface

In [16]:
def neko_lines():
    with open('./neko.txt.cabocha3', encoding='utf-8') as neko_cabocha:
    
        chunks = dict()     # idxをkeyにChunkを格納
        
        for line in neko_cabocha:
            if line[:3] == 'EOS':
                
                # Chunkのリストを返す
                if len(chunks) > 0:

                    # chunksをkeyでソートし、valueのみ取り出し
                    sorted_tuple = sorted(chunks.items(), key=lambda x: x[0])
                    yield list(zip(*sorted_tuple))[1]  #[1]がリストのvalue部分
                    chunks.clear()

                else:
                    yield []
                    
            # 先頭が*の行は係り受け解析結果なので、Chunkを作成
            elif line[0] == '*':

                # Chunkのインデックス番号と係り先のインデックス番号取得
                cols = re.split('\s|D', line)
                idx = int(cols[1]) # Chunkのインデックス番号
                dst = int(cols[2]) # 係り先文節インデックス番号

                # Chunkを生成（なければ）し、係り先のインデックス番号セット
                if idx not in chunks:
                    chunks[idx] = Chunk()
                chunks[idx].dst = dst

                # 係り先のChunkを生成（なければ）し、係り元インデックス番号追加
                if dst != -1:
                    if dst not in chunks:
                        chunks[dst] = Chunk()
                    chunks[dst].srcs.append(idx) # 係り元は複数あるのでappend
                    
            else:
                
                #タブとカンマで分割
                cols = re.split('\t|,', line)

                chunks[idx].morphs.append(Morph(
                        cols[0],    # 表層形(surface)
                        cols[7],    # 基本形(base)
                        cols[1],    # 品詞(pos)
                        cols[2]     # 品詞細分類1(pos1)
                    ))

        raise StopIteration

In [19]:
# 1文ずつリスト作成
for i, chunks in enumerate(neko_lines()):
    
    if i == 100:
        break
    
    for chunk in chunks:
    
        if chunk.dst != -1:
    
            # 記号を除いた表層形をチェック、空なら除外
            src = chunk.output_surface_wo_pct()
            dst = chunks[chunk.dst].output_surface_wo_pct()
            if src != '' and dst != '':
                print('{}:{}\t{}'.format(i, src, dst))    

            

2:吾輩は	猫である
3:名前は	無い
3:まだ	無い
5:どこで	生れたか
5:生れたか	つかぬ
5:とんと	つかぬ
5:見当が	つかぬ
6:何でも	薄暗い
6:薄暗い	所で
6:じめじめした	所で
6:所で	泣いて
6:ニャーニャー	泣いて
6:泣いて	記憶している
6:いた事だけは	記憶している
7:吾輩は	見た
7:ここで	始めて
7:始めて	人間という
7:人間という	ものを
7:ものを	見た
8:しかも	種族であったそうだ
8:あとで	聞くと
8:聞くと	種族であったそうだ
8:それは	種族であったそうだ
8:書生という	人間中で
8:人間中で	種族であったそうだ
8:一番	獰悪な
8:獰悪な	種族であったそうだ
9:この	書生というのは
9:書生というのは	話である
9:時々	捕えて
9:我々を	捕えて
9:捕えて	煮て
9:煮て	食うという
9:食うという	話である
10:しかし	思わなかった
10:その	当時は
10:当時は	なかったから
10:何という	考も
10:考も	なかったから
10:なかったから	思わなかった
10:別段	恐し
10:恐し	思わなかった
10:いとも	思わなかった
11:ただ	載せられて
11:彼の	掌に
11:掌に	載せられて
11:載せられて	持ち上げられた
11:スーと	持ち上げられた
11:持ち上げられた	時
11:時	フワフワした
11:何だか	フワフワした
11:フワフワした	感じが
11:感じが	あったばかりである
12:掌の	上で
12:上で	落ちついて
12:少し	落ちついて
12:落ちついて	見たのが
12:書生の	顔を
12:顔を	見たのが
12:見たのが	人間という
12:いわゆる	人間という
12:人間という	ものの
12:ものの	見始であろう
13:この	時
13:時	ものだと
13:妙な	ものだと
13:ものだと	思った
13:思った	感じが
13:感じが	残っている
13:今でも	残っている
14:第一毛をもって	装飾されべきはずの
14:装飾されべきはずの	顔が
14:顔が	つるつるして
14:つるつるして	薬缶だ
14:まるで	薬缶だ
15:その後	逢ったが
15:猫にも	逢ったが
15:だいぶ	逢ったが
15:逢ったが	ない
15:こんな	片輪には
15:片輪には	ない
15:一度も	